In [10]:
%%capture --no-stderr
%pip install --quiet -U numpy opencv-python

In [13]:
import cv2
import numpy as np

In [18]:
def dark_object_subtraction(image_path, percentile=0.1):

    # Read the image in BGR format
       # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    if image is None:
        raise ValueError(f"Could not read image from {image_path}")

    # Assume image is a NumPy array (BGR or multispectral)
    dark_pixel = np.percentile(image, percentile, axis=(0, 1))
    corrected = np.clip(image - dark_pixel, 0, 255).astype(np.uint8)
    return corrected

In [20]:
# Example usage
corrected_image = dark_object_subtraction("testfiles/DJI_0025-h60.jpeg")
cv2.imwrite("corrected_image.jpeg", corrected_image)

True

Complex DOS and RC

In [26]:

def rigorous_radiometric_correction(image_path, gain, offset, sunelev, edist, Esun, blackadjust=0.01, percentile=0.1):
    """
    Perform rigorous radiometric correction for a multispectral drone image.
    
    Parameters:
      image_path (str): Path to the multispectral image (e.g., from DJI Mavic 3M)
      gain (array-like): Sensor gain for each band (list or numpy array)
      offset (array-like): Sensor offset for each band
      sunelev (float): Sun elevation in degrees
      edist (float): Earth–Sun distance (in astronomical units)
      Esun (array-like): Exo-atmospheric solar irradiance for each band
      blackadjust (float): Adjustment factor (default 0.01) to account for non‑absolute black
      percentile (float): Percentile value (in percent) for dark object estimation (e.g., 0.1 for 0.1th percentile)
      
    Returns:
      np.ndarray: Corrected reflectance image (floating point, typically with values in [0, 1])
    """
    # Load the image (assumed to be multispectral; shape: H x W x bands)
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if image is None:
        raise ValueError(f"Could not read image from {image_path}")
    
    # Convert image to float32 for calculations
    image = image.astype(np.float32)
    
    # Convert digital numbers (DN) to radiance for each band:
    # Radiance = gain * DN + offset
    if image.ndim == 2:
        # Single-channel case:
        radiance = gain * image + offset
    else:
        # Multichannel: assume last dimension corresponds to bands
        h, w, n_bands = image.shape
        radiance = np.empty_like(image)
        for i in range(n_bands):
            radiance[:, :, i] = gain[i] * image[:, :, i] + offset[i]
    
    # Convert radiance to reflectance.
    # Reflectance = (pi * radiance) / (Esun * cos(sun_zenith)) with sun zenith = (90 - sunelev)
    sun_zenith = 90 - sunelev
    cos_sun_zenith = np.cos(np.deg2rad(sun_zenith))
    # Ensure Esun is an array (per band) and apply Earth-Sun distance correction
    Esun = np.array(Esun)
    reflectance = (np.pi * radiance) / (Esun * cos_sun_zenith)
    reflectance = reflectance / (edist ** 2)
    
    # Estimate the dark object value per band on the reflectance image.
    # Using the specified percentile over the spatial dimensions.
    if reflectance.ndim == 2:
        dark_obj = np.percentile(reflectance, percentile, axis=(0, 1))
    else:
        dark_obj = np.percentile(reflectance, percentile, axis=(0, 1))  # dark_obj is per band
    
    # Apply the black adjustment (reduce dark object value by the given fraction)
    dark_obj_adj = dark_obj * (1 - blackadjust)
    
    # Subtract the adjusted dark object value from each band and clip to a valid reflectance range (e.g., [0, 1])
    if reflectance.ndim == 2:
        corrected = np.clip(reflectance - dark_obj_adj, 0, 1)
    else:
        corrected = np.empty_like(reflectance)
        for i in range(reflectance.shape[2]):
            corrected[:, :, i] = np.clip(reflectance[:, :, i] - dark_obj_adj[i], 0, 1)
    
    return corrected


In [ ]:

# Example usage:
if __name__ == "__main__":
    # Replace the following dummy calibration values with your actual sensor metadata:
    image_path = "testfiles/Globhe_SampleData_Multispectral_NDVI.tif"
    # For instance, assume a 4-band multispectral sensor:
    gain = [0.1, 0.11, 0.09, 0.1]         # Example gains per band
    offset = [1, 1, 1, 1]                 # Example offsets per band
    sunelev = 60.0                       # Sun elevation in degrees
    edist = 1.0                          # Earth–Sun distance in AU
    Esun = [1800, 1700, 1600, 1500]        # Exo-atmospheric irradiance per band (example values)
    
    # Run the correction function:
    corrected_reflectance = rigorous_radiometric_correction(
        image_path,
        gain,
        offset,
        sunelev,
        edist,
        Esun,
        blackadjust=0.01,
        percentile=0.1  # Adjust as needed
    )
    
    # Save or further process corrected_reflectance as needed.
    # For visualization, you might convert to 8-bit after scaling reflectance values:
    vis = (corrected_reflectance * 255).astype(np.uint8)
    cv2.imwrite("corrected_reflectance.jpg", vis)


Implementing Geometric correction